<a href="https://colab.research.google.com/github/nishgowda/ml-sandbox/blob/master/lesson7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from google.colab import drive

if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("running on the GPU")
else:
  device = torch.device("cupa")
  print("running on the CPU")

	
REBUILD_DATA = False
drive_path = "/content/drive/My Drive/Images/PetImages"
class DogsVCats():
	IMG_SIZE = 50
	CATS = f"{drive_path}/Cat"
	DOGS = f"{drive_path}/Dog"
	LABELS = {CATS : 0, DOGS : 1}
	training_data = []
	catcount = 0
	dogcount = 0
	
	def make_training_data(self):
		for label in self.LABELS:
			print(label)
			for f in tqdm(os.listdir(label)):
				try:
					path = os.path.join(label, f)
					img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
					img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
					self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
				
					if label == self.CATS:
						self.catcount += 1
					elif label == self.DOGS:
						self.dogcount += 1
				except Exception as e:
						pass
		np.random.shuffle(self.training_data)
		np.save("training_data.npy", self.training_data)
		print("Cats: ", self.catcount)
		print("Dogs: ", self.dogcount) 
if REBUILD_DATA:
	dogvcats = DogsVCats()
	dogvcats.make_training_data()

data_path = "/content/ml-sandbox/pytorch/"
training_data = np.load(f"{data_path}training_data.npy", allow_pickle=True)
#print(len(training_data))
#print(training_data[1])
#plt.imshow(training_data[1][0], cmap = "gray")
#plt.show()


class Net(nn.Module):
        def __init__(self):
                super().__init__()
                self.conv1 = nn.Conv2d(1, 32, 5)
                self.conv2 = nn.Conv2d(32, 64, 5)
                self.conv3 = nn.Conv2d(64, 128, 5)
                
                x = torch.randn(50,50).view(-1,1,50,50)
                self._to_linear = None
                self.convs(x)

                self.fc1 = nn.Linear(self._to_linear, 512)
                self.fc2 = nn.Linear(512, 2)

        def convs(self, x):
                x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
                x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
                x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
                
                #print(x[0].shape)
                if self._to_linear is None:
                        self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
                return x
        
        def forward(self, x):
                x = self.convs(x)
                x = x.view(-1, self._to_linear)
                x = F.relu(self.fc1(x))
                x = self.fc2(x)
                return x

net = Net()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/225.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
#print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]
#print(len(train_X))
#print(len(test_X))

In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
torch.cuda.device_count()

In [14]:
net = Net().to(device)

In [21]:
def train(net):
  BATCH_SIZE = 100
  EPOCHS = 3
  optimizer = optim.Adam(net.parameters(), lr = 0.001)
  loss_function = nn.MSELoss()
  for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
      batch_x = train_X[i:i+BATCH_SIZE].view(-1,1 ,50,50)
      batch_y = train_y[i:i+BATCH_SIZE]
      batch_x, batch_y = batch_x.to(device),batch_y.to(device),
      net.zero_grad()
      outputs = net(batch_x)
      loss = loss_function(outputs, batch_y)
      loss.backward()
      optimizer.step()

    print(f"epoch: {epoch}. Loss {loss}")

train(net)

  3%|▎         | 6/225 [00:00<00:04, 49.54it/s]

epoch: 0. Loss 0.14100100100040436


  3%|▎         | 6/225 [00:00<00:04, 49.61it/s]

epoch: 1. Loss 0.11647967994213104


100%|██████████| 225/225 [00:05<00:00, 43.10it/s]

epoch: 2. Loss 0.097694531083107


In [22]:
def test(net):
  correct = 0 
  total = 0
  with torch.no_grad():
    for i in tqdm(range(len(test_X))):
      real_class = torch.argmax(test_y[i]).to(device)
      net_out = net(test_X[i].view(-1,1,50,50).to(device))[0]
      predicted_class = torch.argmax(net_out)
      if predicted_class == real_class:
        correct += 1
      total += 1
  print("Accuracy: ", round(correct/total, 3))
test(net)

100%|██████████| 2494/2494 [00:02<00:00, 881.73it/s]

Accuracy:  0.748
